In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext


In [ ]:
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [ ]:
# from llama_index.llms.base import HuggingFaceLLM
from llama_index.llms.huggingface import HuggingFaceLLM

/home/kavita/miniconda3/envs/demo/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kavita/miniconda3/envs/demo/lib/python3.11/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [ ]:
from llama_index.core import  ServiceContext


In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
documents=SimpleDirectoryReader("data").load_data()
documents

[Document(id_='2dc8b014-0371-4529-8e22-c16f1c944fca', embedding=None, metadata={'page_label': '1', 'file_name': 'GeM_handbook.pdf', 'file_path': '/home/kavita/Desktop/autogen/LLM/ex2/data/GeM_handbook.pdf', 'file_type': 'application/pdf', 'file_size': 674215, 'creation_date': '2024-07-25', 'last_modified_date': '2024-03-06'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='GeM Handbook\n \nJuly 2018 GOVERNMENT  \nE MARKETPLACE  \n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='1a230c67-2cbf-4ea3-8459-88378135f64b', embedding=None, metadata={'page_label': '2', 'file_name': 'GeM_handbook.pdf', 'file_path': '/home/kavit

In [ ]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="Open-Orca/Mistral-7B-OpenOrca",
    model_name="Open-Orca/Mistral-7B-OpenOrca",
    device_map="cuda",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


OSError: TheBloke/open-llama-3b-v2-wizard-evol-instuct-v2-196k-GGUF does not appear to have a file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt or flax_model.msgpack.

In [ ]:

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

2024-07-25 17:11:33.558463: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-25 17:11:33.910191: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-25 17:11:34.660016: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

/tmp/ipykernel_8540/2250285859.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context=ServiceContext.from_defaults(


In [ ]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x70ac0ad39ed0>, num_workers=None), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x70ac0ad39ed0>, id_func=<function default_id_func at 0x70acf28f3380>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.core.service_context_elements.llama_logger.LlamaLogger object at 0x70abc759e950>, callback_manager=<llama_index

In [ ]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [ ]:
index

In [ ]:

query_engine=index.as_query_engine()

In [ ]:
response=query_engine.query("Which lab of drdo developed technology for HPC disc forgings. Who were partners of DRDO in the same endeavour? ")

/home/kavita/miniconda3/envs/demo/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


In [ ]:
response

Response(response='The Defence Metallurgical Research Laboratory (DMRL), a premier metallurgical laboratory of DRDO at Hyderabad, developed the technology for HPC disc forgings. The partners of DRDO in this endeavour were MIDHANI and HAL (E), Bengaluru.', source_nodes=[NodeWithScore(node=TextNode(id_='dcbaa68b-9d80-4dbd-8eb8-fede77979b95', embedding=None, metadata={'page_label': '10', 'file_name': 'NL_February2024.pdf', 'file_path': '/home/kavita/Desktop/autogen/LLM/ex2/data/NL_February2024.pdf', 'file_type': 'application/pdf', 'file_size': 1706789, 'creation_date': '2024-07-25', 'last_modified_date': '2024-03-06'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='d5b80527-570f-4427-95d1-5a5a3612aa10', node_

In [ ]:
response=query_engine.query("Which drdo lab is working on swarm drone tech?")

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


In [ ]:
response

Response(response='The Defence Research and Development Organisation (DRDO)’s Young Scientist Laboratory for Asymmetric Technologies is working on swarm technologies to strengthen the country’s military capabilities.', source_nodes=[NodeWithScore(node=TextNode(id_='0a25a6a6-d074-4dc7-a804-468fbb1e170d', embedding=None, metadata={'file_path': '/home/kavita/Desktop/autogen/LLM/ex2/data/kk.txt', 'file_name': 'kk.txt', 'file_type': 'text/plain', 'file_size': 29817, 'creation_date': '2024-07-25', 'last_modified_date': '2024-03-06'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='60730fb9-a2ee-4197-b367-a0a0e824eb38', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/home/kavita/Desktop/autogen/LLM/

In [ ]:
query_engine

In [ ]:
print(response)

The Young Scientist Laboratory for Asymmetric Technologies (YSL-AT) of DRDO is working on swarm drone technologies.


In [ ]:
response=query_engine.query("Which patents have been granted to DRDO")

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


In [ ]:
print(response)

The provided text does not mention any specific patents granted to DRDO.


In [ ]:
response=query_engine.query("What is the salary of DRDO JRF and RA?")
print(response)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


The salary of DRDO JRF (Junior Research Fellow) and RA (Research Assistant) is not provided in the given context. To obtain accurate information about their salaries, please refer to the official DRDO website or contact the concerned authorities.


In [ ]:
response=query_engine.query("What is aerial target vehicle of DRDO called as? ")
print(response)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


The aerial target vehicle of DRDO is called ABHYAS.


In [ ]:
response=query_engine.query("Which drdo labs is working on AI ")
print(response)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Centre for Artificial Intelligence & Robotics (CAIR), Bengaluru is a DRDO lab working on AI.


In [ ]:
response=query_engine.query("What is MIDHANI ")
print(response)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


MIDHANI, or Metals and Engineering Consultants Limited, is a public sector undertaking under the Ministry of Defence, India. It is responsible for the procurement, supply, and distribution of raw materials, machinery, and equipment for the Indian defence industry. MIDHANI also provides technical consultancy services and acts as a catalyst for the growth of the Indian defence manufacturing sector.


In [ ]:
response=query_engine.query("Who is Panigrahi")
print(response)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


The provided context does not mention any individual named Panigrahi.


In [ ]:
response=query_engine.query("Which all labs from drdo celebrated RUN FOR UNITY event")
print(response)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


The following DRDO labs celebrated the 'Run for Unity' event:

1. CVRDE, Chennai
2. DESIDOC, Delhi
3. DMRL, Hyderabad
4. DEBEL, Bengaluru
5. DGRE, Chandigarh
6. DIPAS, Delhi
7. EMU, Hyderabad
8. ITM, Mussoorie
9. NMRL, Ambernath
10. NPOL, Kochi


In [ ]:
response=query_engine.query("Which all labs from drdo celebrated RUN FOR UNITY event")
print(response)

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

2994

In [ ]:
response=query_engine.query("Give me event wise list of labs for the following events: RUN FOR UNITY,CYBER Swachhata DIWAS, WORLD QUALITY WEEK")
print(response)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.




For the event "RUN FOR UNITY":
1. Defence Laboratory, Jodhpur (DLJ)
2. Institute for Systems Studies & Analyses (ISSA), Delhi
3. Microwave Tube Research & Development Centre (MTRDC), Bengaluru
4. Naval Science & Technological Laboratory (NSTL), Visakhapatnam
5. Combat Vehicles Research and Development Establishment (CVRDE), Chennai
6. Defence Scientific Information & Documentation Centre (DESIDOC), Delhi
7. Defence Metallurgical Research Laboratory (DMRL), Hyderabad
8. Defence Bionegineering and Electro-medical Laboratory (DEBEL), Bengaluru
9. Defence Geoinformatics Research Establishment (DGRE), Chandigarh
10. Defence Institute of Physiology & Allied Sciences (DIPAS), Delhi

For the event "CYBER Swachhata DIWAS":
(No information provided in the context)

For the event "WORLD QUALITY WEEK":
(No


In [ ]:
response=query_engine.query("Give me event wise list of labs for the following events: CYBER SWACHHATA DIWAS")
print(response)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.




1. DEBEL, Bengaluru
2. DIPAS, Delhi


In [ ]:
### All the class variables are protected
class Car():
    def __init__(self,windows,doors,enginetype):
        self._windows=windows
        self._doors=doors
        self._enginetype=enginetype

In [ ]:
class Truck(Car):
    def __init__(self,windows,doors,enginetype,horsepower):
        super().__init__(windows,doors,enginetype)
        self.horsepowwer=horsepower

In [ ]:
t=Truck(4,4,"desiel",4000)

In [ ]:
dir(t)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_doors',
 '_enginetype',
 '_windows',
 'horsepowwer']

In [ ]:
t._doors=5

In [ ]:
t._doors

5